# Deep Image Stack

![](./assets/2024-06-27_DeepImageStack.png)

## Overview (WIP)
This workflow covers navigating a deep image stack. Often each subsequent frame corresponds to a concurrent time sample, such as the case in neuroscience when conducting calcium imaging with a microscope.



## Prerequisites and Resources (WIP)

| Topic | Type | Notes |
| --- | --- | --- |

## Imports and Configuration

In [1]:
from pathlib import Path
import numpy as np
import xarray as xr
import holoviews as hv
from holoviews.streams import Stream
from holoviews.operation.datashader import rasterize
from hvplot import xarray
import panel as pn

pn.extension(throttled=True)
hv.extension('bokeh')

## Loading and Inspecting the Data

Let's read the data in chunks, emulating a situation where a dataset is too large to fit into memory. Utilizing the `chunks` parameter in `xr.open_dataset` is crucial for efficient data handling with large datasets, as it enables Dask to process the data in manageable portions.

In [2]:
DATA_PATH = "../data/real_miniscope_uint8.zarr"
# DATA_PATH = "data/sim_miniscope.zarr"

# Open the dataset from the Zarr storage
ds = xr.open_dataset(
    DATA_PATH,
    engine = 'zarr',
    # chunks = {'frame': 'auto', 'height':-1, 'width':-1},  # chunk by sets of complete frames
)
ds.compute()

<xarray.Dataset> Size: 722MB
Dimensions:   (frame: 2000, height: 480, width: 752)
Coordinates:
  * frame     (frame) int64 16kB 0 1 2 3 4 5 6 ... 1994 1995 1996 1997 1998 1999
  * height    (height) int64 4kB 0 1 2 3 4 5 6 7 ... 473 474 475 476 477 478 479
  * width     (width) int64 6kB 0 1 2 3 4 5 6 7 ... 745 746 747 748 749 750 751
Data variables:
    varr_ref  (frame, height, width) uint8 722MB 0 0 2 2 2 2 2 ... 0 0 0 2 2 2 2

From the output above, we can see that the actual DataArray that we are looking for is called '`varr_ref` - let's go ahead and get a handle on that.

In [3]:
da = ds['varr_ref'] #varr_ref #sim_miniscope
da

<xarray.DataArray 'varr_ref' (frame: 2000, height: 480, width: 752)> Size: 722MB
[721920000 values with dtype=uint8]
Coordinates:
  * frame    (frame) int64 16kB 0 1 2 3 4 5 6 ... 1994 1995 1996 1997 1998 1999
  * height   (height) int64 4kB 0 1 2 3 4 5 6 7 ... 473 474 475 476 477 478 479
  * width    (width) int64 6kB 0 1 2 3 4 5 6 7 ... 745 746 747 748 749 750 751

## Data Visualization

Visualizing calcium imaging data effectively is key to extracting meaningful insights. We introduce various visualization approaches to cater to different analysis needs.

We will start with one-liner viewer and then proceed to a more advanced application with enhanced interactive features and exposed controls, at the expense of code complexity.

### Basic: Quick App with Time-Projected Image

We can use one line with hvPlot for a quick inspection of the deep image stack. To the left of this, we'll also put a max-projection of the image stack frame over time as a reference of all potential neurons that fluorescence at some point in the movie. Max projecting over time just means keeping the maximum value from the time-stack in each position of the width/height coordinates.

In [4]:
# # Create the max time-projected image
# max_proj_time = da.max('frame').compute().astype(np.float32)
# img_max_proj_time = max_proj_time.hvplot.image(x='width', y='height', title='Max Over Time', cmap = "magma", aspect = da.sizes['width'] / da.sizes['height'],)

# # Create the Deep Image Stack App (this line is sufficient to create a basic app)
# img_stack_viewer = da.hvplot.image(groupby="frame", title='Deep Image Stack', cmap = "viridis", aspect = da.sizes['width'] / da.sizes['height'])

# basic_img_stack_app = img_max_proj_time + img_stack_viewer
# basic_img_stack_app

This was a quick way to see one frame at a time! But it looks like there are a lot of fluorescing neurons in the left `'Max Over Time'` image that came from somewhere in the data. How do we visually locate which frames in the `'Deep Image Stack'` (right) a neuron is flourescing in?

Well, our data array is a three-dimensional volume, so if we also had **side-view** of the volume, we might be able to locate the frames where a particular region is fluorescing.

### Advanced App with Side-Views, Swipe Overlay, and Annotations Linked to Timeseries View

The Side-View App builds on the Basic App with added functionality:

1. **Continuous Playback:** Player widget for continuous playback, along with controls for step-by-step examination of the image stack.
2. **Side Views** Aggregated side views for display over 'deep' dimension.
3. **Synchronized Frame Indicators** Frame markers synchronized with the playback and x,y range of the main image stack view.
4. **Slider Overlay Alpha** Slider widget to adjust transparency of max-over-time overlay for direct comparison.
5. **Scale Bar**
6. TODO: **Interactive Annotations** Enables direct annotation of regions of interest within the visual interface utilizing the HoloNote package.
7. TODO: **Timeseries of Annotations** Annotations are synchronized to an adjacent stacked timeseries plot.

In [5]:
def plot_image(value):
    return hv.Image(da.sel(frame=value), kdims=["width", "height"]).opts(
        frame_height=da.sizes['height'],
        frame_width=da.sizes['width'],
        cmap = "Viridis",
        # title = "Deep Image Stack [Max Over Time]",
        tools=['hover', 'crosshair'],
        toolbar='right',
        scalebar=True,
        scalebar_unit=("µm", "m"), # each data bin is roughly 1 µm
        apply_hard_bounds=True,
        scalebar_opts={
        'background_fill_alpha': 0.5,
        'border_line_color': None,
        'bar_length': 0.10,
        }
    )

# Create a player widget
video_player = pn.widgets.Player(
    length =len(da.coords["frame"]),
    interval = 100,  # ms
    value = 950, # start frame
    width=da.sizes['width'],
    height=90,
    loop_policy="loop",
)

# Create the main frame-wise view (height by width)
main_view = hv.DynamicMap(pn.bind(plot_image, video_player))

# right-side view: frame by height
right_data = da.mean(["width"]).persist()
right_view = rasterize(
    hv.Image(right_data, kdims=["frame", "height"]).opts(
        frame_height=da.sizes['height'],
        frame_width=175,
        colorbar=False,
        title="Side View",
        toolbar='right',
        cmap = "Viridis",
        tools=['crosshair', 'hover'],
        yaxis='right',
        axiswise=True,
        apply_hard_bounds=True,
    )
)


# top-side view: width by frame 
top_data = da.mean(["height"]).persist()
top_view = rasterize(
    hv.Image(top_data, kdims=["width", "frame"]).opts(
        frame_height=175,
        frame_width=da.sizes['width'],
        colorbar=False,
        toolbar='right',
        cmap = "Viridis",
        title= "Top View",
        tools=['crosshair', 'hover'],
        xaxis='top',
        axiswise=True,
        apply_hard_bounds=True,
    )
)

# frame-indicator lines on side view plots
line_opts = dict(color="red", line_width=7, line_alpha=.4)

xyrange_stream = hv.streams.RangeXY(source=main_view)

def plot_hline(value, x_range, y_range):
    if x_range == None:
        x_range = [int(da.width[0].values), int(da.width[-1].values)]
    return hv.Segments((x_range[0], value, x_range[1], value)).opts(axiswise=True)

def plot_vline(value, x_range, y_range):
    if y_range == None:
        y_range = [int(da.height[0].values), int(da.height[-1].values)]
    return hv.Segments((value, y_range[0], value, y_range[1])).opts(axiswise=True)

dmap_hline = hv.DynamicMap(pn.bind(plot_hline, video_player), streams=[xyrange_stream]).opts(
    **line_opts
)

dmap_vline = hv.DynamicMap(pn.bind(plot_vline, video_player), streams=[xyrange_stream]).opts(
    **line_opts
)

# Create the max time-projected image
max_proj_time = da.max('frame').compute().astype(np.float32)
img_max_proj_time = hv.Image(
    max_proj_time, ['width', 'height'], label='Max Over Time').opts(
    frame_height=da.sizes['height'],
    frame_width=da.sizes['width'],
    cmap='magma',
)

# Bind the player widget's value directly to the markdown pane's object parameter
frame_markdown = pn.pane.Markdown()
frame_markdown.object = pn.bind(lambda value: f'# frame: {value}', video_player.param.value)

alpha_slider = pn.widgets.FloatSlider(start=0, end=1, step=.001, value=0.5, name='Alpha of Max Over Time', align='center')

# Lay out the app views
video_player.margin = (0, 0, 0, -70)  # center widget with main
alpha_slider.margin = (0, 0, 20, -170)  # center widget with main
img_stack_app = pn.Column(
    (top_view * dmap_hline).opts(axiswise=True),
    pn.Row(
        main_view * img_max_proj_time.apply.opts(alpha=alpha_slider), # TODO JS link this instead
           (right_view * dmap_vline).opts(axiswise=True), margin=0), 
    pn.Column(
    alpha_slider,
    pn.Row(video_player, frame_markdown), align='center')
)

# img_stack_app.servable()


# With Annotations (WIP)

In [7]:
from holonote.annotate import Annotator
from holonote.app import PanelWidgets
from holonote.app.tabulator import AnnotatorTabulator

# TODO: Create a secondary subcoordinate_y plot that is synced with the holonote table of annotations

# # Update plot based on annotation selection
# def plot_timeseries_by_select(indices):
#     if indices:
#         h1, h2, w1, w2 = indices[0]["start[height]"], indices[0]["end[height]"], indices[0]["start[width]"], indices[0]["end[width]"]
#         ds_sel = da.sel(height=slice(h1, h2), width=slice(w1, w2)).mean(
#             ["height", "width"]
#         )
#         time_series.object = hv.Curve(ds_sel)

# # Update plot based on annotation creation
# def plot_timeseries_by_stream(bounds):
#     if bounds:
#         h1, h2, w1, w2 = bounds
#         ds_sel = da.sel(height=slice(h1, h2), width=slice(w1, w2))
#         time_series.object = hv.Curve(ds_sel.mean(["height", "width"])).opts(
#         )

# Annotation setup
annotator = Annotator({"height": float, "width": float}, fields=["type"])
annotator.groupby = "type"
annotator_widgets = pn.Column(PanelWidgets(annotator), AnnotatorTabulator(annotator))

time_series = pn.pane.HoloViews()

# # Update timeseries plot on annotation selection
# pn.bind(plot_timeseries_by_select, annotator.param.selected_indices, watch=True)

# # Update timeseries plot on annotation creation
# display = annotator.get_display("height", "width")
# box_stream = display._edit_streams[0]
# box_stream.source = main_view
# pn.bind(plot_timeseries_by_stream, box_stream.param.bounds, watch=True)

eventi = []
def print_event(event):
    global eventi
    eventi = event
    print(event)

annotator.on_event(print_event)

img_stack_app_annotator = pn.Row(pn.Column(annotator_widgets,
                                           # annotator * top_view * dmap_hline,
                                           pn.Row(annotator * main_view,
                                                  # annotator * right_view * dmap_vline, 
                                                 ), 
                                           pn.Row(video_player, frame_markdown),
                                          ),
                                 time_series)

img_stack_app_annotator.servable()

event type create 
creating... 
add to widget 
Traceback (most recent call last): 
 File "/Users/droumis/opt/miniconda3/envs/neuro-deep-image-stack/lib/python3.12/site-packages/holoviews/plotting/util.py", line 293, in get_plot_frame 
 return map_obj[key] 
 ~~~~~~~^^^^^ 
 File "/Users/droumis/opt/miniconda3/envs/neuro-deep-image-stack/lib/python3.12/site-packages/holoviews/core/spaces.py", line 1216, in __getitem__ 
 val = self._execute_callback(*tuple_key) 
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ 
 File "/Users/droumis/opt/miniconda3/envs/neuro-deep-image-stack/lib/python3.12/site-packages/holoviews/core/spaces.py", line 983, in _execute_callback 
 retval = self.callback(*args, **kwargs) 
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ 
 File "/Users/droumis/opt/miniconda3/envs/neuro-deep-image-stack/lib/python3.12/site-packages/holoviews/core/spaces.py", line 552, in __call__ 
 return self.callable() 
 ^^^^^^^^^^^^^^^ 
 File "/Users/droumis/opt/miniconda3/envs/neuro-deep-image-stack/lib/python3.12/site-packages/holoviews/util/__init__.py", line 1038, in dynamic_operation 
 key, obj = resolve(key, kwargs) 
 ^^^^^^^^^^^^^^^^^^^^ 
 File "/Users/droumis/opt/miniconda3/envs/neuro-deep-image-stack/lib/python3.12/site-packages/holoviews/util/__init__.py", line 1027, in resolve 
 return key, map_obj[key] 
 ~~~~~~~^^^^^ 
 File "/Users/droumis/opt/miniconda3/envs/neuro-deep-image-stack/lib/python3.12/site-packages/holoviews/core/spaces.py", line 1219, in __getitem__ 
 self._cache(tuple_key, val) 
 File "/Users/droumis/opt/miniconda3/envs/neuro-deep-image-stack/lib/python3.12/site-packages/holoviews/core/spaces.py", line 1292, in _cache 
 self[key] = val 
 ~~~~^^^^^ 
 File "/Users/droumis/opt/miniconda3/envs/neuro-deep-image-stack/lib/python3.12/site-packages/holoviews/core/ndmapping.py", line 566, in __setitem__ 
 self._add_item(key, value, update=False) 
 File "/Users/droumis/opt/miniconda3/envs/neuro-deep-image-stack/lib/python3.12/site-packages/holoviews/core/ndmapping.py", line 165, in _add_item 
 self._item_check(dim_vals, data) 
 File "/Users/droumis/opt/miniconda3/envs/neuro-deep-image-stack/lib/python3.12/site-packages/holoviews/core/ndmapping.py", line 984, in _item_check 
 raise AssertionError(f"{self.__class__.__name__} must only contain one type of object, not both {type(data).__name__} and {self.type.__name__}.") 
AssertionError: DynamicMap must only contain one type of object, not both Overlay and NdOverlay. 
 
event_info(type='create', index='60c0debaa3a742819c56ba66179678ed', region={'width': (361.7890625, 447.33203125), 'height': (200.7578125, 276.26171875)}, fields={'type': 'A', 'uuid': '60c0debaa3a742819c56ba66179678ed'})

Row
    [0] Column
        [0] Column
            [0] Column
                [0] MultiSelect(name='Visible', stylesheets=['\n            option:aft...])
                [1] Column
                    [0] TextInput(name='type', placeholder='type')
                [2] Row
                    [0] Button(description=Tooltip(id='92919d57-a608-..., name='✓', width=20)
                    [1] Spacer(width=10)
                    [2] RadioButtonGroup(description=Tooltip(id='02104e92-edca-..., name='Mode', options=['+', '-', '✏'], value='+', width=90)
                    [3] Spacer(width=10)
                    [4] Button(description=Tooltip(id='0c6d2faf-f5d7-..., name='↺', width=20)
                    [5] Button(description=Tooltip(id='b30d7929-5d4e-..., name='▲', width=20)
            [1] Tabulator(buttons={'delete': '<i c...}, show_index=False, value=Empty DataFrame
Columns: [...)
        [1] Row
            [0] HoloViews(DynamicMap, sizing_mode='fixed')
        [2] Row
            [0] Player(end=1999, height=90, interval=100, loop_policy='loop', margin=(0, 0, 0, -70), sizing_mode='fixed', value=950, value_throttled=950, width=752)
            [1] Markdown(str)
    [1] HoloViews(None)

In [ ]:

# annotator = Annotator({"height": float, "width": float}, fields=["type"],
#                       connector=SQLiteDB(filename=':memory:'))
# annotator.groupby = "type"
# annotator_widgets = PanelWidgets(annotator)

# plot = annotator * hv.Image([], ['width', 'height'])
# pn.Column(annotator_widgets, plot).servable()



In [1]:
from holonote.annotate import Annotator
from holonote.app import PanelWidgets
import panel as pn; pn.extension()
import holoviews as hv; hv.extension('bokeh')
from holonote.annotate.connector import SQLiteDB
import numpy as np

annotator = Annotator(
        {"x": float, "y": float},
        fields=["description"],
        connector=SQLiteDB(filename=':memory:')
    )
annotator.groupby = "description"
bounds = (-1, -1, 1, 1)
data = np.array([[0, 1], [1, 0]])
img = hv.Image(data, kdims=["x", "y"], bounds=bounds)

plot = annotator * img
hv.render(plot)

annotator.set_regions(x=(-0.15, 0.15), y=(-0.25, 0.25))
annotator.add_annotation(description="Test")


Traceback (most recent call last):
  File "/Users/droumis/opt/miniconda3/envs/neuro-deep-image-stack/lib/python3.12/site-packages/holoviews/plotting/util.py", line 293, in get_plot_frame
    return map_obj[key]
           ~~~~~~~^^^^^
  File "/Users/droumis/opt/miniconda3/envs/neuro-deep-image-stack/lib/python3.12/site-packages/holoviews/core/spaces.py", line 1216, in __getitem__
    val = self._execute_callback(*tuple_key)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/droumis/opt/miniconda3/envs/neuro-deep-image-stack/lib/python3.12/site-packages/holoviews/core/spaces.py", line 983, in _execute_callback
    retval = self.callback(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/droumis/opt/miniconda3/envs/neuro-deep-image-stack/lib/python3.12/site-packages/holoviews/core/spaces.py", line 552, in __call__
    return self.callable()
           ^^^^^^^^^^^^^^^
  File "/Users/droumis/opt/miniconda3/envs/neuro-deep-image-stack/lib/python3.12/site-pa

In [6]:
plot

:DynamicMap   []
   :Overlay
      .Image.I      :Image   [x,y]   (z)
      .Image.II     :Image   [x,y]   (z)
      .NdOverlay.I  :NdOverlay   [description]
         :Rectangles   [start[x],start[y],end[x],end[y]]   (description,__selected__)
      .Rectangles.I :Rectangles   [x0,y0,x1,y1]
      .Path.I       :Path   [x,y]

In [5]:
print(plot)

:DynamicMap   []
   :Overlay
      .Image.I      :Image   [x,y]   (z)
      .Image.II     :Image   [x,y]   (z)
      .NdOverlay.I  :NdOverlay   [description]
         :Rectangles   [start[x],start[y],end[x],end[y]]   (description,__selected__)
      .Rectangles.I :Rectangles   [x0,y0,x1,y1]
      .Path.I       :Path   [x,y]


:DynamicMap   []
   :Overlay
      .Image.I      :Image   [width,height]   (z)
      .Image.II     :Image   [width,height]   (z)
      .NdOverlay.I  :NdOverlay   [type]
         :Rectangles   [start[width],start[height],end[width],end[height]]   (type,__selected__)
      .Rectangles.I :Rectangles   [x0,y0,x1,y1]
      .Path.I       :Path   [x,y]

In [6]:
assert isinstance(plot, hv.NdOverlay), "plot should be an instance of hv.NdOverlay"

AssertionError: plot should be an instance of hv.NdOverlay

In [ ]:
# color = ['red', 'blue']

# def update_style(value):
#     md = pn.pane.Markdown('# test')
#     md.stylesheets = [f"""
#             h1 {{
#             background: {color[value]};
#             }}
#             """]
#     return md

# slider = pn.widgets.IntSlider(start=0, value=0, end=1)

# pn.Column(pn.bind(update_style, slider), slider)

In [18]:
# color = ['red', 'blue']

# def update_style(value):
#     md = pn.pane.Markdown('# test')
#     md.stylesheets = [f"""
#             h1 {{
#             background: {color[value]};
#             }}
#             """]
#     return md

# slider = pn.widgets.IntSlider(start=0, value=0, end=1)

# pn.Column(pn.bind(update_style, slider), slider)

Column
    [0] ParamFunction(function, _pane=Markdown, defer_load=False)
    [1] IntSlider()

In [12]:
# md.stylesheets = ["""
#             h1 {
#             background: blue;
#             }
#             """]

### using pn.bind

In [ ]:
def curve_plot(value):
    curves = {}
    for i,row in df.iterrows():
        h1, h2, w1, w2 = row[["start[height]", "end[height]", "start[width]", "end[width]"]]
        ds_sel = da.sel(height=slice(h1, h2), width=slice(w1, w2))
        for n in range(value):
            curves[f"{n} {row['type']} {i[:6]}"] = hv.Curve(ds_sel.mean(["height", "width"]), group=row['type']).opts(subcoordinate_y=True)
    return hv.NdOverlay(curves, ['curve']).opts(width=800, show_legend=False)

slider = pn.widgets.IntSlider(start=1, value=2, end=3)
plot = pn.bind(curve_plot, slider)

pn.Column(slider, plot)

### using dmap

In [ ]:
def curve_plot(value):
    curves = {}
    for i,row in df.iterrows():
        h1, h2, w1, w2 = row[["start[height]", "end[height]", "start[width]", "end[width]"]]
        ds_sel = da.sel(height=slice(h1, h2), width=slice(w1, w2))
        for n in range(value):
            curves[f"{n} {row['type']} {i[:6]}"] = hv.Curve(ds_sel.mean(["height", "width"]), group=row['type']).opts(subcoordinate_y=True)
    return hv.NdOverlay(curves, ['curve']).opts(width=800, show_legend=False)

slider = pn.widgets.IntSlider(start=1, value=10, end=10)
plot = hv.DynamicMap(pn.bind(curve_plot, slider))

pn.Column(slider, plot)

In [ ]:
h1 = 191.124141421
h2 = 226
w1 = 297
w2 = 371

ds_sel = da.sel(height=slice(h1, h2), width=slice(w1, w2))
hv.Curve(ds_sel.mean(["height", "width"]))

# super simple example

In [3]:
# ensure there is no .db in the cwd

from holonote.annotate import Annotator
from holonote.app.tabulator import AnnotatorTable
from holonote.app import PanelWidgets
import panel as pn; pn.extension()
import holoviews as hv; hv.extension('bokeh')
from holonote.annotate.connector import SQLiteDB

annotator = Annotator({"height": float, "width": float}, fields=["type"],
                      connector=SQLiteDB(filename=':memory:'))
annotator.groupby = "type"
annotator_widgets = pn.Column(PanelWidgets(annotator), AnnotatorTable(annotator))

pn.Column(annotator_widgets, annotator * hv.Image([], ['width', 'height'])).servable()

Column
    [0] Column
        [0] Column
            [0] MultiSelect(name='Visible', stylesheets=['\n            option:aft...])
            [1] Column
                [0] TextInput(name='type', placeholder='type')
            [2] Row
                [0] Button(description=Tooltip(id='56f7f145-ebfc-..., name='✓', width=20)
                [1] Spacer(width=10)
                [2] RadioButtonGroup(description=Tooltip(id='fc5ce493-7604-..., name='Mode', options=['+', '-', '✏'], value='+', width=90)
                [3] Spacer(width=10)
                [4] Button(description=Tooltip(id='dfc505b8-d598-..., name='↺', width=20)
                [5] Button(description=Tooltip(id='f784baea-fe28-..., name='▲', width=20)
        [1] Tabulator(buttons={'delete': '<i c...}, show_index=False, value=Empty DataFrame
Columns: [...)
    [1] HoloViews(DynamicMap, height=300, sizing_mode='fixed', width=300)

TODO:
- check the visible field or groupby field in event, if not yet in options, update it.
- the if_event def should check the type of event, compare existing to the potentially new set of options

In [ ]:
# from holonote.annotate import Annotator
# from holonote.app import PanelWidgets

# def tmp_plot_image(value):
#     return hv.Image(da.sel(frame=1), kdims=["width", "height"]).opts(
#         frame_height=100,
#         frame_width=150,
#         cmap = "Viridis",
#         tools=['hover', 'crosshair'],
#         toolbar='right',
# )

# # Create a player widget
# video_player = pn.widgets.Player(
#     length = 100,
#     interval = 250,  # ms
#     value = 950, # start frame
# )

# dy_img = hv.DynamicMap(pn.bind(tmp_plot_image, video_player))

# annotator = Annotator({"height": float, "width": float}, fields=["type"])
# annotator_widgets = PanelWidgets(annotator)

# pn.Column(annotator_widgets, annotator * dy_img, video_player)

In [ ]:
annotator.add_annotation(type='A')

Issues:
- ~~Creating an annotation: AssertionError: DynamicMap must only contain one type of object, not both Overlay and NdOverlay.~~
- Visibility GUI not updating
- add linked timeseries plot
- ~~Side View not showing annotation on correct axis~~
- ~~multiple toolbars with swipe (causing frame misalignment)~~
- ~~how to show multiple colorbars with swipe?~~
- freehanddraw polygon to box annotation, with polygon saved as a displayable field?